<a href="https://colab.research.google.com/github/Hasib0077/nlp1/blob/main/new_tokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset
raw_dataset=load_dataset("code_search_net","python")

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [3]:
raw_dataset["train"]


Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [4]:
raw_dataset["train"][123456]["whole_func_string"]

'def core_properties(self):\n        """\n        Instance of |CoreProperties| holding the read/write Dublin Core\n        document properties for this presentation. Creates a default core\n        properties part if one is not present (not common).\n        """\n        try:\n            return self.part_related_by(RT.CORE_PROPERTIES)\n        except KeyError:\n            core_props = CorePropertiesPart.default()\n            self.relate_to(core_props, RT.CORE_PROPERTIES)\n            return core_props'

In [7]:
def get_training_corpus():
  return(
      raw_dataset["train"][i:i+1000]["whole_func_string"]
      for i in range(0,len(raw_dataset["train"]),1000)
  )

traing_corpus=get_training_corpus()

In [9]:
#training

from transformers import AutoTokenizer
old_tokenizer=AutoTokenizer.from_pretrained("gpt2")

example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [11]:
tokenizer=old_tokenizer.train_new_from_iterator(traing_corpus,52000)
tokens=tokenizer.tokenize(example)
tokens

['d',
 'e',
 'f',
 'Ġ',
 'a',
 'd',
 'd',
 '_',
 'n',
 'u',
 'm',
 'b',
 'e',
 'r',
 's',
 '(',
 'a',
 ',',
 'Ġ',
 'b',
 ')',
 ':',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 '"',
 '"',
 '"',
 'A',
 'd',
 'd',
 'Ġ',
 't',
 'h',
 'e',
 'Ġ',
 't',
 'w',
 'o',
 'Ġ',
 'n',
 'u',
 'm',
 'b',
 'e',
 'r',
 's',
 'Ġ',
 '`',
 'a',
 '`',
 'Ġ',
 'a',
 'n',
 'd',
 'Ġ',
 '`',
 'b',
 '`',
 '.',
 '"',
 '"',
 '"',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'r',
 'e',
 't',
 'u',
 'r',
 'n',
 'Ġ',
 'a',
 'Ġ',
 '+',
 'Ġ',
 'b']

In [17]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

82
36


In [21]:
tokenizer.save_pretrained("code-search-net-tokenizer")
from huggingface_hub import notebook_login
notebook_login()